In [1]:
import findspark
findspark.init()

import pyspark
# import pyspark.pandas as ps
import datetime as dt

from src.emulator import generate_transactions_table
from src.emulator import generate_customer_profiles_table
from src.emulator import generate_terminal_profiles_table
from src.emulator import get_list_terminals_within_radius
from src.emulator import generate_dataset, add_frauds

In [2]:
spark = (
    pyspark.sql.SparkSession.builder
        .appName("OTUS")
        .master("yarn")
        .config("spark.executor.memory", "1g")
        .config("spark.driver.memory", "1g")
        .getOrCreate()
)

In [3]:
(customer_profiles_table_df, terminal_profiles_table_df, transactions_df)=\
    generate_dataset(n_customers = 5000, 
                     n_terminals = 10000, 
                     nb_days=250, 
                     start_date=dt.date(year=2021, month=1, day=1).isoformat(), 
                     r=5)

transactions_sdf=spark.createDataFrame(transactions_df) 
transactions_sdf.write.mode("append").parquet("hdfs:///card-fraud-detection/transactions.parquet")

terminal_profiles_table_sdf=spark.createDataFrame(terminal_profiles_table_df) 
terminal_profiles_table_sdf.write.mode("append").parquet("hdfs:///card-fraud-detection/customer_profiles_table.parquet")

terminal_profiles_table_sdf=spark.createDataFrame(terminal_profiles_table_df) 
terminal_profiles_table_sdf.write.mode("append").parquet("hdfs:///card-fraud-detection/terminal_profiles_table.parquet")    

Time to generate customer profiles table: 0.056s
Time to generate terminal profiles table: 0.053s
Time to associate terminals to customers: 2.4s
Time to generate transactions: 1.3e+02s


/usr/lib/spark/python/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Py4JJavaError: An error occurred while calling o87.parquet.
: org.apache.hadoop.security.AccessControlException: Permission denied: user=ubuntu, access=WRITE, inode="/":hdfs:hadoop:drwxr-xr-x
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:399)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:255)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:193)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1896)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1880)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkAncestorAccess(FSDirectory.java:1839)
	at org.apache.hadoop.hdfs.server.namenode.FSDirMkdirOp.mkdirs(FSDirMkdirOp.java:59)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.mkdirs(FSNamesystem.java:3252)
	at org.apache.hadoop.hdfs.server

In [4]:
transactions_sdf.count()

2395956

In [5]:
spark.stop()

In [6]:
!echo $PYTHON_PATH